# West Africa 1990-2021
#### LINEAR INVERSION & GRID SEARCH

Jean-Joel Legre  URSeismo,2022

Reference:

Nienke Brinkman Interactive_SI_script copy.py   (https://github.com/nienkebrinkman/SS_MTI) 

Brinkman, Nienke, et al. "First focal mechanisms of marsquakes." Journal of Geophysical Research: Planets 126.4 (2021): e2020JE006546.



In [ ]:
from os.path import join as pjoin
from os.path import exists as exist
from os import listdir as lsdir
from os import makedirs
from pathlib import Path
from obspy.geodetics.base import gps2dist_azimuth
from collections import OrderedDict

import obspy
from obspy.taup import TauPyModel
from obspy.core import UTCDateTime
from obspy import read,read_inventory,read_events

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from mpl_toolkits.basemap import Basemap
from obspy.imaging.beachball import beach
from obspy.imaging.beachball import aux_plane

import instaseis
import toml
    
import SS_MTI

import sys
sys.path.append("/scratch/tolugboj_lab/softwares/SS_MTI")
sys.path.append("/scratch/tolugboj_lab/softwares/SS_MTI/Scripts_MT_Structure")
sys.path.append("/scratch/tolugboj_lab/softwares/SS_MTI/Scripts")
sys.path.append("/scratch/tolugboj_lab/softwares/SS_MTI/SS_MTI_Scripts")
#from SS_MTI_Scripts import DataGetterWafr
from SS_MTI_Scripts import InversionWafr
from SS_MTI_Scripts import ForwardWafr
from SS_MTI_Scripts import PostProcessingWafr

import pygmt
import geopandas

In [ ]:
events = read_events('Output/Input_SSMTI/MTInputCatalog/WAMCA_Quakes.xml')
WAMCA_input =  pd.read_csv('Output/Input_SSMTI/MTInputCatalog/WAMCA_input2.csv') 
WAMCA_input.sort_values(by='origin',inplace = True, ascending=False)
print(events)
WAMCA_input.reset_index(drop=True, inplace=True)
WAMCA_input

#### Parameters for Inversion 

In [ ]:
phases = ["P","S"]
components = ["Z","T"]
tstars = [0.4, 0.2]
zerophase = True
amplitude_correction = ["PZ","ST"]
t_pre = [1, 1]
t_post = [12,12]
weights = [[10,1],[10,1]]
start_weight_len = 5.0
ylims = [1, 1]

#### Forward Modeling (Synthetic Waveforms)

In [ ]:
VelModel = "IASP91_2s" , # ak135f_no_mud, PREM, or IASP91

path_VelModels = "data/velocity_model/"
npz_file = pjoin(path_VelModels,("%s.npz")%VelModel)   

# Syngine Database of Synthetic Seismograms
db = instaseis.open_db("syngine://%s"%VelModel)

# Start inversion

In [ ]:
solutions_folder = "Output/Solutions/Final"     
Path(solutions_folder).mkdir(parents=True,exist_ok=True)
BBstrike1=[]
BBdip1=[]
BBrake1=[]

BBstrike2=[]
BBdip2=[]
BBrake2=[]

BBdepth=[]

for i,row in WAMCA_input.iloc[:].iterrows():
    
    otime = row['origin']
    evindice = row['indices'] 
    print(otime)
    events[i].origin_time = UTCDateTime(events[i].origins[0].time)
    
    if (otime == events[i].origin_time) :
        
        #Event        
        events[i].name = row['folders']
        fmin = row['Freqmin']
        fmax = row['Freqmax']

        Color_GS = row['BBColors']
        
        output_folder = pjoin(solutions_folder,events[i].name)
        Path(output_folder).mkdir(parents=True,exist_ok=True)
        
        #Additional information needed for events                 
        events[i].distance = row['epi_distance_deg']
        events[i].baz = int(row['back_azimuths'])
        events[i].az = int(row['azimuths'])
        events[i].latitude = row['lat']
        events[i].longitude = row['lon']
        events[i].depth = int(row['depth'])

        #P and S
        P_S_arr=row['Arrival_PS'].strip("[]")
        P_S_arr=P_S_arr.split(",")
        events[i].picks=[float(P_S_arr[0]),float(P_S_arr[1])]
        phase_corrs = [float(row['P_corr']) , float(row['S_corr'])] #arrival time correction 

        # Receiver's information
        lat_stat = row['stlat']
        lon_stat = row['stlon']

        #Observed Data
        sta_selected = row['station']
        try:
            path_to_waveform = "Output/Input_SSMTI/InputWaveforms/%s/%s*.mseed"%(events[i].name,sta_selected)
            obs_data = obspy.read(path_to_waveform)
            events[i].waveforms = obs_data

            samplerate = obs_data[2].stats.sampling_rate
            dt = (1/samplerate)
            print(f"dt is {dt}")

            #Input for Instaseis
            rec = instaseis.Receiver(latitude= lat_stat , longitude= lon_stat)

            #--------------------------------------------------------------------------------------------------------------
            # Forward Modelling Method
            forward_method = "INSTASEIS"
            #extra_phases to plot
            extra_phases = ["PP","pP", "sP", "sS", "SS"]

            if forward_method == "INSTASEIS":
                fwd = ForwardWafr.Instaseis(
                            instaseis_db=db,
                            taup_model= VelModel[0][:-3],
                            or_time= events[i].origin_time,
                            dt=dt,
                            start_cut= 10.0,   
                            end_cut=1200.0,
                )
            elif forward_method == "REFLECTIVITY":
                 fwd = SS_MTI.Forward.reflectivity()
            else:
                 raise ValueError(
                       "forward_method can be either INSTASEIS or REFLECTIVITY in [FORWARD] of .toml file"
                        )
            #--------------------------------------------------------------------------------------------------------------
            misfit_method = "L2"

            if misfit_method == "L2":
                misfit = SS_MTI.Misfit.L2(weights=weights, start_weight_len=start_weight_len, dt=dt)
            elif misfit_method == "CC":
                misfit = SS_MTI.Misfit.CC(shift_samples=128)
            elif misfit_method == "POL":
                misfit = SS_MTI.Misfit.Pol(
                          components=components, start_weight_len=start_weight_len, weights=weights, dt=dt
                        )
            else:
                raise ValueError("misfit can be L2, CC or POL in [MISFIT]")

            #--------------------------------------------------------------------------------------------------------------    

            #Grid of Source Parameters for Inversion
            evdepth = events[i].depth 
            
            depths = np.arange(evdepth-int(evdepth)+1,21,1)
            
            if evdepth in np.arange(2,22,2):
                depths_plot = np.arange(2,22,2)
            else:
                depths_plot = np.arange(1,20,2)
                
            strikes = np.arange(0, 360, 5)
            dips = np.arange(0, 90, 5)
            rakes = np.arange(-180, 180, 5)
            bazs = [events[i].baz] 

            #Inversion
            output_folder_GS = pjoin(output_folder,"Result_Grid_Search")
            Path(output_folder_GS).mkdir(parents=True,exist_ok=True)

            output_folder_LI = pjoin(output_folder,"Result_Linear_Inversion")
            Path(output_folder_LI).mkdir(parents=True,exist_ok=True)

            InversionWafr.Grid_Search_run(
                fwd=fwd,
                misfit=misfit,
                event=events[i],
                rec=rec,
                phases= phases,     
                components=components,
                t_pre=t_pre,
                t_post=t_post,
                depths=depths,
                strikes=strikes,
                dips=dips,
                rakes=rakes,
                phase_corrs=phase_corrs,
                tstars=tstars,
                fmin=fmin,
                fmax=fmax,
                zerophase=zerophase,
                list_to_correct_M0=amplitude_correction,
                plot=True,
                plot_extra_phases=extra_phases,
                color_plot= Color_GS,
                Ylims=ylims,
                Parallel=True,
                output_folder=output_folder_GS
            )
            InversionWafr.Direct(
                fwd=fwd,
                misfit=misfit,
                event=events[i],
                rec=rec,
                phases=phases,
                components=components,
                phase_corrs=phase_corrs,
                t_pre=t_pre,
                t_post=t_post,
                depths=depths,
                tstars=tstars,
                fmin=fmin,
                fmax=fmax,
                zerophase=zerophase,
                plot=True,
                plot_extra_phases=extra_phases,
                color_plot="dimgrey",
                Ylims=ylims,
                Parallel=True,
                output_folder=output_folder_LI
            )


            #Post Processing-----------------------------------------------------------------------------------------------

            PostProcessingWafr.post_waveform_plotting(
                event=events[i],
                h5_file_folder=output_folder_GS,
                method="GS",
                misfit_name=misfit.name,
                misfit_weight_len=misfit.start_weight_len,
                fwd=fwd,
                rec=rec,
                phases=phases,
                components=components,
                t_pre=t_pre,
                t_post=t_post,
                depths=depths,
                phase_corrs=phase_corrs,
                fmin=fmin,
                fmax=fmax,
                zerophase=zerophase,
                tstars=tstars,
                plot_extra_phases=extra_phases,
                Ylims=ylims,
                color_plot=Color_GS,
            ) 

            
            # misfit vs depth analysis
            Moho_d = 24
            DOF = sum([int((x + y) / dt) for x, y in zip(t_pre, t_post)]) 

            try:
                fig,BB_preferred,sol_depth,allFP1 = PostProcessingWafr.plot_misfit_vs_depth(
                                                    baz=events[i].baz,
                                                    save_paths=[output_folder],
                                                    event_name=events[i].name,
                                                    DOF=DOF,
                                                    depths=depths,
                                                    evindice = evindice,
                                                    misfit_name=misfit.name,
                                                    veloc_model=fwd.veloc_name,
                                                    true_depth=float(evdepth),
                                                    Moho=Moho_d,
                                                    fmin=fmin,
                                                    fmax=fmax,
                                                    amount_of_phases=len(phases),
                                                    color_plot=Color_GS,
                )
                plt.tight_layout()
                plt.savefig(
                    pjoin(
                        output_folder,
                        f"Misfit_vs_Depth_{events[i].name}_{fmin}_{fmax}_{misfit.name}_{fwd.veloc_name}_{events[i].baz}.pdf",
                    ),
                    dpi=600,
                )

                plt.close()
            
            except Exception as e:
                exception_type, exception_object, exception_traceback = sys.exc_info()
                line_number = exception_traceback.tb_lineno 
                print(line_number)
                print(exception_object)
            
            # plot_phases_vs_depth
            try:
                start_depth_range = [sol_depth,sol_depth]  
                end_depth_range = [sol_depth,sol_depth]   

                if sol_depth == 19 :
                    depths = np.arange(evdepth-int(evdepth)+1,21,1)
                    
                if sol_depth == 20 :
                    depths = np.arange(evdepth-int(evdepth)+1,22,1)    

                fig = PostProcessingWafr.plot_phases_vs_depth(
                    h5_file_folder=output_folder_GS,
                    method="GS",
                    misfit_name=misfit.name,
                    fwd=fwd,
                    event=events[i],
                    evindice = evindice,
                    rec=rec,
                    sol_depth =sol_depth,
                    phases=phases,
                    color_plot=Color_GS,
                    components=components,
                    t_pre=t_pre,
                    t_post=t_post,
                    depths= depths, 
                    allBBs = allFP1,
                    phase_corrs=phase_corrs,
                    fmin=fmin,
                    fmax=fmax,
                    zerophase=zerophase,
                    tstars=tstars,
                    pref_depth_start=start_depth_range,
                    pref_depth_end=end_depth_range,
                )
                plt.tight_layout()
                plt.savefig(
                    pjoin(
                        output_folder,
                        f"Full_PhaseTracking_{events[i].name}_{fmin}_{fmax}_{misfit.name}_{fwd.veloc_name}.pdf",
                    ),
                    dpi=600,
                )
                plt.close()
            except Exception as e:
                exception_type, exception_object, exception_traceback = sys.exc_info()
                line_number = exception_traceback.tb_lineno 
                print(line_number)
                print(exception_object)
            
            
            if sol_depth in np.arange(2,22,2):
                depths_plot = np.arange(2,22,2)
            else:
                depths_plot = np.arange(1,20,2)
            
                
            try:
                fig = PostProcessingWafr.plot_misfit_vs_depth(
                                                    baz=events[i].baz,
                                                    save_paths=[output_folder],
                                                    event_name=events[i].name,
                                                    DOF=DOF,
                                                    depths=depths_plot,
                                                    evindice = evindice,
                                                    misfit_name=misfit.name,
                                                    veloc_model=fwd.veloc_name,
                                                    true_depth=float(evdepth),
                                                    Moho=Moho_d,
                                                    fmin=fmin,
                                                    fmax=fmax,
                                                    amount_of_phases=len(phases),
                                                    color_plot=Color_GS,
                )
                plt.tight_layout()
                plt.savefig(
                    pjoin(
                        output_folder,
                        f"Misfit_vs_Depth_{events[i].name}_{fmin}_{fmax}_{misfit.name}_{fwd.veloc_name}_{events[i].baz}.pdf",
                    ),
                    dpi=600,
                )

                plt.close()
            
            except Exception as e:
                exception_type, exception_object, exception_traceback = sys.exc_info()
                line_number = exception_traceback.tb_lineno 
                print(line_number)
                print(exception_object)
                
            #Fault Plane Solutions: F1 and F2
            s2,d2,r2= aux_plane(BB_preferred[0],BB_preferred[1],BB_preferred[2])


            #Source_Uncertainty
            fig_sdr = PostProcessingWafr.Source_Uncertainty(
                h5_file_folder=output_folder_GS,
                h5_file_folder_direct=output_folder_LI,
                event_name=events[i].name,
                method="GS",
                misfit_name=misfit.name,
                fwd=fwd,
                phases=phases,
                components=components,
                depths=depths,
                BB_planes=FPs,
                sol_depth = sol_depth,
                DOF=DOF,
                fmin=fmin,
                fmax=fmax,
                color= Color_GS
            )

            fig_sdr.tight_layout()
            fig_sdr.savefig(
                pjoin(
                    output_folder,
                    f"Uncertainties_SDR_{events[i].name}_{fmin}_{fmax}_{misfit.name}_{fwd.veloc_name}.pdf",
                ),
                dpi=600,
            )
            plt.close(fig_sdr)            


            #polarization plot
            PostProcessingWafr.polarization(
                h5_file_folder=output_folder_GS,
                h5_file_folder_direct=output_folder_LI,
                event=events[i],
                method="GS",
                misfit_name=misfit.name,
                fwd=fwd,
                phases=phases,
                components=components,
                depths=depths,
                DOF=DOF,
                fmin=fmin,
                fmax=fmax,
            )

            BBstrike1.append(BB_preferred[0])
            BBdip1.append(BB_preferred[1])
            BBrake1.append(BB_preferred[2])
        
            BBstrike2.append(s2)
            BBdip2.append(d2)
            BBrake2.append(r2)
        
            BBdepth.append(sol_depth)
            
               
        except Exception as e:
                exception_type, exception_object, exception_traceback = sys.exc_info()
                line_number = exception_traceback.tb_lineno 
                print(line_number)
                print(exception_object)
        
                       

In [ ]:
Table_sources = WAMCA_input[["country","origin","indices","lon","lat","mag","station","stlon","stlat","BBColors"]]
Table_sources

In [ ]:
Table_sources.insert(6, "depth",BBdepth, True)
Table_sources.insert(10, "strike1",BBstrike1, True)
Table_sources.insert(11, "dip1",BBdip1, True)
Table_sources.insert(12, "rake1",BBrake1, True)
Table_sources.insert(13, "strike2",BBstrike2, True)
Table_sources.insert(14, "dip2",BBdip2, True)
Table_sources.insert(15, "rake2",BBrake2, True)
#Table_sources.to_csv('Output/Input_SSMTI/Focal_Mechanisms/Table_sources.csv',index=False)
Table_sources

In [ ]:
HCA_input =  pd.read_csv('Output/Input_SSMTI/MTInputCatalog/HistoricalMomentTensors_.csv')
Table_sources = pd.read_csv('Output/Input_SSMTI/Focal_Mechanisms/Table_sources.csv')
Table_sources

## Geology

In [ ]:
WAC = np.loadtxt('Output/Input_SSMTI/Geology/Cratons/West_Africa_Craton_Coordinates.txt')
SMC = np.loadtxt('Output/Input_SSMTI/Geology/Cratons/Sahara_Metacraton.txt')
CC = np.loadtxt('Output/Input_SSMTI/Geology/Cratons/Congo_Craton.txt')

Countries_name = pd.read_csv('Output/Input_SSMTI/Geology/CountriesNames.csv')


#Shear zones

DataShearzoneAlg1 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Algeria_1.csv')
DataShearzoneAlg2 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Algeria_2.csv')
DataShearzoneCA1 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Central_africa_1.csv')
DataShearzoneCA2 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Central_africa_2.csv')
DataShearzoneCA3 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Central_africa_3.csv')
DataShearzoneCA4 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Central_africa_4.csv')
DataShearzoneCA5 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Central_africa_5.csv')

DataShearzoneGuinea = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Guinea.csv')
DataShearzoneIZSZ = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/IZSZ.csv')
DataShearzoneKSZ = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/KSZ.csv')
DataShearzoneMau_bot = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Mauritania_bottom.csv')
DataShearzoneMau_top = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Mauritania_top.csv')
DataShearzoneNorth_Mali = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/North_Mali.csv')

DataShearzoneRSZ1 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/RSZ_1.csv')
DataShearzoneRSZ2 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/RSZ_2.csv')
DataShearzoneRSZ3 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/RSZ_3.csv')
DataShearzoneRSZ4 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/RSZ_4.csv')

DataShearzoneSah_bot = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/SaharaMet_Bot.csv')
DataShearzoneSah_top = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/SaharaMet_top.csv')
DataShearzoneSass = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Sassandra.csv')
DataShearzoneSen = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/Senegal.csv')
DataShearzoneSouth_Ni_1 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/South_Niger_branch_1.csv')
DataShearzoneSouth_Ni_2 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/shearzones_csv/South_Niger_branch_2.csv')



#basins
DatabasinBenue_Through = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Benue_Through.csv')
DatabasinBida_Basin_1 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Bida_Basin_1.csv')
DatabasinBida_Basin_2 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Bida_Basin_2.csv')
Databasinchad_left = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/chad_left.csv')
Databasinchad_right = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/chad_right.csv')
DatabasinGoo_basin = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Goo_basin.csv')
DatabasinGoo_basin_2 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Goo_basin_2.csv')
DatabasinTermit_basin_1 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Termit_basin_1.csv')
DatabasinTermit_basin_2 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Termit_basin_2.csv')
DatabasinTermit_basin_3 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Termit_basin_3.csv')
DatabasinTermit_basin_4 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Termit_basin_4.csv')
DatabasinTermit_basin_5 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Termit_basin_5.csv')
DatabasinTermit_basin_6 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins2csv/Termit_basin_6.csv')


DatabasinArda_left = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/Arda_left.csv')
DatabasinGourmaAulacogen = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/GourmaAulacogen.csv')
DatabasinGourmaAulacogen2 = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/GourmaAulacogen2.csv')
DatabasinHank = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/Hank.csv')
DatabasinHodh = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/Hodh.csv')
DatabasinRichat_left = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/Richat_left.csv')
DatabasinRichat_right = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/Richat_right.csv')
DatabasinTambaoura_Bottom = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/Tambaoura_Bottom.csv')
DatabasinTambaoura_top = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/Tambaoura_top.csv')
DatabasinYB_left = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/YB_left.csv')
DatabasinYB_right = pd.read_csv('Output/Input_SSMTI/Geology/shearzones_and_basins/basins3csv/YB_right.csv')


## Plots

In [ ]:
topo_data = '@earth_relief_30s'

In [ ]:
fig = pygmt.Figure()

# 
with fig.subplot(nrows=1, ncols=1, figsize=("25c", "20c"), autolabel="a)"):
    
    pygmt.makecpt(
        cmap='gray',
        series='-30000/3000/500',
        continuous=True
    )
    fig.grdimage(
            grid=topo_data,
            region=[-25, 27, -15, 27],
            projection="M0/5/25c",
            shading=True,
            frame="a", 
            panel=[0, 0],
    )
    
    
    fig.coast(region=[-25, 27, -15, 27],projection="M0/5/25c",borders="1/0.2p,dimgray",
              shorelines="1/0.5p",resolution="h")   
 
    #Geology
    fig.plot(projection="M0/5/25c",x=WAC[:,1],y=WAC[:,0],pen="2p,black",label="Craton Boundary")
    fig.plot(projection="M0/5/25c",x=SMC[:,1],y=SMC[:,0],pen="2p,black")
    fig.plot(projection="M0/5/25c",x=CC[:,1],y=CC[:,0],pen="2p,black")
    
    #shear zones
    fig.plot(projection="M0/5/25c",x=DataShearzoneAlg1["Longitude"].values,y=DataShearzoneAlg1["Latitude"].values,pen="2p,blue,4p_4p") 
    fig.plot(projection="M0/5/25c",x=DataShearzoneAlg2["Longitude"].values,y=DataShearzoneAlg2["Latitude"].values,pen="2p,blue,4p_4p") 
    fig.plot(projection="M0/5/25c",x= DataShearzoneCA1["Longitude"].values,y= DataShearzoneCA1["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x=  DataShearzoneCA2["Longitude"].values,y= DataShearzoneCA2["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneCA3["Longitude"].values,y= DataShearzoneCA3["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneCA4["Longitude"].values,y= DataShearzoneCA4["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneCA5["Longitude"].values,y= DataShearzoneCA5["Latitude"].values,pen="2p,blue,4p_4p")

    fig.plot(projection="M0/5/25c",x= DataShearzoneGuinea["Longitude"].values, y= DataShearzoneGuinea["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x=DataShearzoneIZSZ["Longitude"].values, y=DataShearzoneIZSZ["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x=DataShearzoneKSZ["Longitude"].values, y=DataShearzoneKSZ["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x=DataShearzoneMau_bot["Longitude"].values, y=DataShearzoneMau_bot["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneMau_top["Longitude"].values, y= DataShearzoneMau_top["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x=DataShearzoneNorth_Mali["Longitude"].values, y=DataShearzoneNorth_Mali["Latitude"].values,pen="2p,blue,4p_4p")

    fig.plot(projection="M0/5/25c",x= DataShearzoneRSZ1["Longitude"].values,y= DataShearzoneRSZ1["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneRSZ2["Longitude"].values ,y= DataShearzoneRSZ2["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneRSZ3["Longitude"].values ,y= DataShearzoneRSZ3["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneRSZ4["Longitude"].values, y= DataShearzoneRSZ4["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x=DataShearzoneSah_bot["Longitude"].values ,y=DataShearzoneSah_bot["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneSah_top["Longitude"].values, y= DataShearzoneSah_top["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneSass["Longitude"].values , y= DataShearzoneSass["Latitude"].values,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= (DataShearzoneSen["Longitude"].values)+0.8, y= (DataShearzoneSen["Latitude"].values)-0.5,pen="2p,blue,4p_4p")
    fig.plot(projection="M0/5/25c",x= DataShearzoneSouth_Ni_1["Longitude"].values,y= DataShearzoneSouth_Ni_1["Latitude"].values,pen="2p,blue,4p_4p") 
    fig.plot(projection="M0/5/25c",x= DataShearzoneSouth_Ni_2["Longitude"].values,y= DataShearzoneSouth_Ni_2["Latitude"].values,pen="2p,blue,4p_4p")
    
    fig.plot(projection="M0/5/25c",x= DatabasinGourmaAulacogen["longitude"].values,y= DatabasinGourmaAulacogen["latitude"].values,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x= DatabasinGourmaAulacogen["longitude"].values,y= 0.2 + (DatabasinGourmaAulacogen["latitude"].values),pen="2p,gold")
    fig.plot(projection="M0/5/25c",x= DatabasinGourmaAulacogen2["longitude"].values,y= (DatabasinGourmaAulacogen2["latitude"].values) + 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x= DatabasinGourmaAulacogen2["longitude"].values,y= DatabasinGourmaAulacogen2["latitude"].values,pen="2p,gold")
    
    
    
    fig.plot(projection="M0/5/25c",x= DatabasinTermit_basin_1 ["longitude"].values,y= DatabasinTermit_basin_1["latitude"].values,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x= DatabasinTermit_basin_1 ["longitude"].values,y= (DatabasinTermit_basin_1 ["latitude"].values)- 0.2 ,pen="2p,gold")
    fig.plot(projection="M0/5/25c",x= DatabasinTermit_basin_2["longitude"].values,y= (DatabasinTermit_basin_2["latitude"].values) + 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=DatabasinTermit_basin_2["longitude"].values,y= DatabasinTermit_basin_2["latitude"].values,pen="2p,gold")
    fig.plot(projection="M0/5/25c",x= DatabasinTermit_basin_3["longitude"].values,y= (DatabasinTermit_basin_3["latitude"].values) + 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=DatabasinTermit_basin_3["longitude"].values,y= DatabasinTermit_basin_3["latitude"].values,pen="2p,gold")
    fig.plot(projection="M0/5/25c",x= DatabasinTermit_basin_4["longitude"].values,y= (DatabasinTermit_basin_4["latitude"].values) - 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=DatabasinTermit_basin_4["longitude"].values,y= DatabasinTermit_basin_4["latitude"].values,pen="2p,gold")
    fig.plot(projection="M0/5/25c",x= DatabasinTermit_basin_5["longitude"].values,y= (DatabasinTermit_basin_5["latitude"].values) + 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=DatabasinTermit_basin_5["longitude"].values,y= DatabasinTermit_basin_5["latitude"].values,pen="2p,gold")
    fig.plot(projection="M0/5/25c",x= DatabasinTermit_basin_6["longitude"].values,y= (DatabasinTermit_basin_6["latitude"].values) + 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=DatabasinTermit_basin_6["longitude"].values,y= DatabasinTermit_basin_6["latitude"].values,pen="2p,gold")
    fig.plot(projection="M0/5/25c",x= DatabasinBenue_Through["longitude"].values,y= (DatabasinBenue_Through["latitude"].values) + 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=DatabasinBenue_Through["longitude"].values,y= DatabasinBenue_Through["latitude"].values,pen="2p,gold")
    
    fig.plot(projection="M0/5/25c",x= DatabasinBida_Basin_1["longitude"].values,y= (DatabasinBida_Basin_1["latitude"].values) + 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=DatabasinBida_Basin_1["longitude"].values,y= DatabasinBida_Basin_1["latitude"].values,pen="2p,gold")
    fig.plot(projection="M0/5/25c",x= DatabasinBida_Basin_2["longitude"].values,y= (DatabasinBida_Basin_2["latitude"].values) - 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=DatabasinBida_Basin_2["longitude"].values,y= DatabasinBida_Basin_2["latitude"].values,pen="2p,gold")
    
    fig.plot(projection="M0/5/25c",x= Databasinchad_left["longitude"].values,y= (Databasinchad_left["latitude"].values) + 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=Databasinchad_left["longitude"].values,y= Databasinchad_left["latitude"].values,pen="2p,gold")
    fig.plot(projection="M0/5/25c",x= Databasinchad_right ["longitude"].values,y= (Databasinchad_right ["latitude"].values) + 0.2,pen="5p,gold,10p_10p")
    fig.plot(projection="M0/5/25c",x=Databasinchad_right ["longitude"].values,y= Databasinchad_right ["latitude"].values,pen="2p,gold")
    
    
    
    fig.text(projection="M0/5/25c",text="WAC", x=-7, y=20,font="30p,black")
    fig.text(projection="M0/5/25c",text="SMC", x=15.5, y=13,font="30p,black")
    fig.text(projection="M0/5/25c",text="CC", x=23, y=2.5,font="30p,black")
    #Subframes
    fig.plot(projection="M0/5/25c",data=np.array([[-15,5,-8,16]]), style='r+s', pen="3p,black")
    fig.plot(projection="M0/5/25c",data=np.array([[-15,5,-8,16]]), style='r+s', pen="3p,white,20p_20p")
    fig.plot(projection="M0/5/25c",data=np.array([[-2,4,4,8]]), style='r+s', pen="3p,black")
    fig.plot(projection="M0/5/25c",data=np.array([[-2,4,4,8]]), style='r+s', pen="3p,white,10p_10p")
    
    #blackframe
    fig.plot(projection="M0/5/25c",data=np.array([[-16,4.2,12,24.5]]), style='r+s', pen="7p,black")
    fig.plot(projection="M0/5/25c",data=np.array([[-16,4.2,12,24.5]]), style='r+s', pen="2p,white")
    
    
    
    #Geography
    xname = Countries_name["Longitude"].values
    yname = Countries_name["Latitude"].values
    zname = list(Countries_name["Names"].values)
    fig.text(projection="M0/5/25c",text=zname, x= xname, y= yname,font="10p,dimgray")
    

    for i,row in HCA_input.iloc[:].iterrows():  
        strike= row["Strike1"]
        dip= row['Dip1']
        rake= row["Slip1"]
        magnitude= row["Mag"]
        depth= row["Depth"]
        G= "dimgrey"
        longitude = row["lon"]
        latitude = row["lat"]
        
        if i == 2 :
            plot_latitude =latitude-3
            plot_longitude =longitude-1.5
            
            fig.plot(projection="M0/5/25c",x=[longitude,plot_longitude],y=[latitude,plot_latitude],pen="2p,black")
 
            longitude=plot_longitude
            latitude=plot_latitude 

            
        if  i==3 :
            plot_latitude =latitude -3
            plot_longitude =longitude+1.5 
            
            fig.plot(projection="M0/5/25c",x=[longitude,plot_longitude],y=[latitude,plot_latitude],pen="2p,black")
 
            longitude=plot_longitude
            latitude=plot_latitude 
            
        if i == 0 :

            plot_latitude =10
            plot_longitude =-20
            
            fig.plot(projection="M0/5/25c",x=[longitude,plot_longitude],y=[latitude,plot_latitude],pen="2p,black")
    
            longitude=plot_longitude
            latitude=plot_latitude 
            
        focal_mechanism = dict(strike=strike, dip=dip, rake=rake, magnitude=magnitude)
         
        # pass the focal mechanism data to meca in addition to the scale and event location
        fig.meca(focal_mechanism, scale="1c", longitude=longitude, latitude=latitude,
                 depth=depth,G=G,projection="M0/5/25c")
        #
        if i == 2 :
            fig.text(projection="M0/5/25c",text="M6.1-06/1939", x= plot_longitude-3.7, y= plot_latitude+0.5,font="10p,black")
   
        if  i==3 :
            fig.text(projection="M0/5/25c",text="M6.5-06/1939", x= plot_longitude+3.5, y= plot_latitude+1,font="10p,black")

            
        if i == 0 :
            fig.text(projection="M0/5/25c",text="M6.3-12/1983", x= plot_longitude-2.5, y= plot_latitude-1.7,font="10p,black")

        
        

    for i,row in Table_sources.iloc[:].iterrows():

        # store focal mechanisms parameters in a dict
        strike= round(row["strike1"],0)
        dip= round(row["dip1"],0)
        rake= round(row["rake1"],0)
        magnitude= row["mag"]
        depth= row["depth"]
        G= row["BBColors"]
        longitude = row["lon"]
        latitude = row["lat"]
        focal_mechanism = dict(strike=strike, dip=dip, rake=rake, magnitude=magnitude)

        # pass the focal mechanism data to meca in addition to the scale and event location
        fig.meca(focal_mechanism, scale="1c", longitude=longitude, latitude=latitude,
                 depth=depth,G=G,projection="M0/5/25c")
    for m in zip([35.10,347.83],[49.24,47.33],[68.11,112.23],[10,10],[4.5,4.6],[10.62,5.80],[19.62,22.42],["forestgreen","red"]):


        # store focal mechanisms parameters in a dict
        strike= m[0]
        dip= m[1]
        rake= m[2]
        magnitude= m[4]
        depth= m[3]
        G= m[7]
        longitude = m[5]
        latitude = m[6]
        focal_mechanism = dict(strike=strike, dip=dip, rake=rake, magnitude=magnitude)

        # pass the focal mechanism data to meca in addition to the scale and event location
        fig.meca(focal_mechanism, scale="1c", longitude=longitude, latitude=latitude,
                 depth=depth,G=G,projection="M0/5/25c")
        
    for i_,rad in enumerate(zip([0.6,0.8,1,1.2],["M3-4","M4-5","M5-6","M6-7"])):
        rad_=rad[0]
        label_=rad[1]
        fig.plot(
                projection="M0/5/25c",x=-30, y=5,style=f"c{rad_}c", color="white",pen="1p,black",label=label_
            )    
        
    fig.legend(projection="M0/5/25c")
    
# 
fig.shift_origin(xshift="w+2c")

#
with fig.subplot(nrows=2, ncols=1, figsize=("8c", "20c"), autolabel="b)"):
    pygmt.makecpt(
        cmap='gray',
        series='-30000/3000/500',
        continuous=True
    )
    fig.grdimage(
            grid=topo_data,
            region=[-15, -7, 5, 15],
            projection="M8c",
            shading=True,
            frame="a", 
            panel=[0, 0],
    )
    fig.coast(region=[-15, -7, 5, 15],projection="M8c",borders="1/0.2p,dimgray",
              shorelines="1/0.5p",resolution="h")

    fig.text(projection="M8c",text=zname, x= xname, y= yname,font="10p,dimgray")
    
    for i,row in Table_sources.iloc[:].iterrows():

        # store focal mechanisms parameters in a dict
        strike= round(row["strike1"],0)
        dip= round(row["dip1"],0)
        rake= round(row["rake1"],0)
        magnitude= row["mag"]
        depth= row["depth"]
        G= row["BBColors"]
        longitude = row["lon"]
        latitude = row["lat"]
        
        if i == 0 :

            plot_latitude =latitude +2
            plot_longitude =longitude+1
            
            y=[latitude,plot_latitude]
            fig.plot(projection="M8c",x=[longitude,plot_longitude],y=[latitude,plot_latitude],pen="2p,black")
            longitude=plot_longitude
            latitude=plot_latitude 
    
        if longitude >-15 and longitude <-8 and latitude > 5 and latitude < 14:
        
            focal_mechanism = dict(strike=strike, dip=dip, rake=rake, magnitude=magnitude)

            # pass the focal mechanism data to meca in addition to the scale and event location
            fig.meca(focal_mechanism, scale="1c", longitude=longitude, latitude=latitude,
                     depth=depth,G=G,projection="M8c")
     
    
    
    for i,row in HCA_input.iloc[:].iterrows():  
            strike= row["Strike1"]
            dip= row['Dip1']
            rake= row["Slip1"]
            magnitude= row["Mag"]
            depth= row["Depth"]
            G= "dimgrey"
            longitude = row["lon"]
            latitude = row["lat"]

            if i == 0 :
                plot_latitude =latitude+0.5
                plot_longitude =longitude-0.5

                fig.plot(projection="M8c",x=[longitude,plot_longitude],y=[latitude,plot_latitude],pen="2p,black")
                fig.plot(
                projection="M8c",x=longitude, y=latitude,style="c0.2c", color="black",pen="1p,black"
                )
                longitude=plot_longitude
                latitude=plot_latitude 

    
            focal_mechanism = dict(strike=strike, dip=dip, rake=rake, magnitude=magnitude)

            # pass the focal mechanism data to meca in addition to the scale and event location
            fig.meca(focal_mechanism, scale="1c", longitude=longitude, latitude=latitude,
                     depth=depth,G=G,projection="M8c")
            #

    

    #
    pygmt.makecpt(
            cmap='gray',
            series='-30000/3000/500',
            continuous=True
        )
    fig.grdimage(
            grid=topo_data,
            region=[-2,4,4,8],
            projection="M8c",
            shading=True,
            frame="a", 
            panel=[1, 0],
    )
    fig.coast(region=[-2, 4, 4,8],projection="M8c",borders="1/0.2p,dimgray",
             shorelines="1/0.5p",resolution="h")
    
    fig.text(projection="M8c",text=zname[19], x= 0.30, y= 6.30,font="10p,dimgray")
    
    for i,row in Table_sources.iloc[:].iterrows():

        # store focal mechanisms parameters in a dict
        strike= round(row["strike1"],0)
        dip= round(row["dip1"],0)
        rake= round(row["rake1"],0)
        magnitude= row["mag"]
        depth= row["depth"]
        G= row["BBColors"]
        longitude = row["lon"]
        latitude = row["lat"]
        

        if longitude >-2 and longitude <4 and latitude > 4 and latitude < 8:
            if i == 3 :
                plot_latitude =latitude +0.75
                plot_longitude =longitude-0.75
                
                y=[latitude,plot_latitude]
                fig.plot(projection="M8c",x=longitude,y=latitude,style="c0.2c", color="black",pen="1p,black")
                fig.plot(projection="M8c",x=[longitude,plot_longitude],y=[latitude,plot_latitude],pen="2p,black")
                longitude=plot_longitude
                latitude=plot_latitude  
        
            focal_mechanism = dict(strike=strike, dip=dip, rake=rake, magnitude=magnitude)
            # pass the focal mechanism data to meca in addition to the scale and event location
            fig.meca(focal_mechanism, scale="1c", longitude=longitude, latitude=latitude,
                     depth=depth,G=G,projection="M8c")
            
            
    for i,row in HCA_input.iloc[:].iterrows():  
            strike= row["Strike1"]
            dip= row['Dip1']
            rake= row["Slip1"]
            magnitude= row["Mag"]
            depth= row["Depth"]
            G= "dimgrey"
            longitude = row["lon"]
            latitude = row["lat"]

            if i == 2 :
                plot_latitude =latitude-0.5
                plot_longitude =longitude-0.5

                fig.plot(projection="M8c",x=[longitude,plot_longitude],y=[latitude,plot_latitude],pen="2p,black")
                fig.plot(
                projection="M8c",x=longitude, y=latitude,style="c0.2c", color="black",pen="1p,black"
                )
                longitude=plot_longitude
                latitude=plot_latitude 


            if  i==3 :
                plot_latitude =latitude 
                plot_longitude =longitude+0.9

                fig.plot(projection="M8c",x=[longitude,plot_longitude],y=[latitude,plot_latitude],pen="2p,black")
                fig.plot(
                projection="M8c",x=longitude, y=latitude,style="c0.2c", color="black",pen="1p,black"
                )
                
                longitude=plot_longitude
                latitude=plot_latitude 
                fig.plot(
                projection="M8c",x=longitude, y=latitude,style="c0.2c", color="black",pen="1p,black"
                )

    

            focal_mechanism = dict(strike=strike, dip=dip, rake=rake, magnitude=magnitude)

            # pass the focal mechanism data to meca in addition to the scale and event location
            fig.meca(focal_mechanism, scale="1c", longitude=longitude, latitude=latitude,
                     depth=depth,G=G,projection="M8c")
            #


output ="Output/Figures/Figure_4a.pdf"   
fig.savefig(output, transparent=False, crop=True, anti_alias=True, show=True)
fig.show()

In [ ]:
plt.style.use('default')
plt.rcParams["figure.figsize"] = (30,10)

#depth vs lon
source_depth = Table_sources["depth"].values
source_lon = Table_sources["lon"].values
source_color = list(Table_sources["BBColors"].values)
source_indices = Table_sources["indices"].values

source_depth=np.append(source_depth,[10,10])
source_lon=np.append(source_lon,[10.62,5.80])
source_color.append("forestgreen")
source_color.append("red")
source_indices=np.append(source_indices,[5,7])


ax=plt.subplot()
for sr in zip(source_lon, source_depth,source_color,source_indices):
    
    if sr[3] in [5,7]:
        marker = 's'
    else:
        marker = 'o'
        
        
    ax.scatter(sr[0],sr[1],4000,marker=marker,color=sr[2])
    ax.text(sr[0],sr[1],sr[3],ha="center",va="center",color="white",fontsize=40,fontweight="bold")
for d in [5,10,15,20]:
    ax.hlines(d,-15,15,linewidth=4,linestyle="--",color="black")

ax.text(-14,-1.5,"West",fontsize=40,fontweight="bold",ha="center")
ax.text(14,-1.5,"East",fontsize=40,fontweight="bold",ha="center")
ax.text(11.1,9.5,"?",fontsize=50,fontweight="bold",va="bottom")
ax.text(6.3,9.5,"?",fontsize=50,fontweight="bold",va="bottom")

ax.set_xlim(-15,15)
ax.set_ylim(0,25)
for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(3)
ax.tick_params(axis="both", labelsize=35,direction="out",length=12,width=3)
ax.invert_yaxis()
ax.set_xlabel("Longitudes($^\circ$)",fontsize=40,fontweight="bold")
ax.set_ylabel("Depths(km)",fontsize=40,fontweight="bold")

output ="Output/Figures/"
plt.savefig(pjoin(output,"Figure_4c.pdf"),dpi=600,)

plt.show()
